# **Identify Customer Segments For Online Retail With The Use Of K-Means Clustering**

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# **Data**

In [2]:
# load the data
data = pd.read_csv('Online_Retail.csv') 
data
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [3]:
#information about data
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
#describe the data
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
#count number of missing values in which columns
nan_counts = data.isnull().sum()
nan_counts

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
print(data['StockCode'].unique())
print(data['InvoiceDate'].unique())
print(data['CustomerID'].unique())
print(data['Country'].unique())

['85123A' '71053' '84406B' ... '90214U' '47591b' '23843']
['12/1/2010 8:26' '12/1/2010 8:28' '12/1/2010 8:34' ... '12/9/2011 12:31'
 '12/9/2011 12:49' '12/9/2011 12:50']
[17850. 13047. 12583. ... 13298. 14569. 12713.]
['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


In [7]:
print(data['StockCode'].value_counts())
print(data['InvoiceDate'].value_counts())
print(data['CustomerID'].value_counts())
print(data['Country'].value_counts())

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
21431        1
22275        1
17001        1
90187A       1
72759        1
Name: StockCode, Length: 4070, dtype: int64
10/31/2011 14:41    1114
12/8/2011 9:28       749
12/9/2011 10:03      731
12/5/2011 17:24      721
6/29/2011 15:58      705
                    ... 
9/28/2011 9:57         1
9/28/2011 9:45         1
3/16/2011 12:13        1
3/16/2011 12:22        1
3/16/2011 15:58        1
Name: InvoiceDate, Length: 23260, dtype: int64
17841.0    7983
14911.0    5903
14096.0    5128
12748.0    4642
14606.0    2782
           ... 
15070.0       1
15753.0       1
17065.0       1
16881.0       1
16995.0       1
Name: CustomerID, Length: 4372, dtype: int64
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002


# Fill missing values

In [8]:
# Convert InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Fill missing CustomerID values based on InvoiceDate
data['CustomerID'] = data.groupby('InvoiceDate')['CustomerID'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else -1))


# Fill missing Description with a placeholder

data['Description'] = data.groupby('InvoiceNo')['Description'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No Description'))


data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
​


In [9]:
nan_counts = data.isnull().sum()
nan_counts

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [11]:
data = data.drop_duplicates()

In [12]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [13]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,536641.000000,536641.000000,536641.000000
mean,9.620029,4.632656,11714.510477
std,219.130156,97.233118,6635.218066
min,-80995.000000,-11062.060000,-1.000000
25%,1.000000,1.250000,12471.000000
50%,3.000000,2.080000,14428.000000
75%,10.000000,4.130000,16326.000000
max,80995.000000,38970.000000,18287.000000


In [14]:

# Calculate TotalSpend
data['TotalSpend'] = data['Quantity'] * data['UnitPrice']

# Aggregate data to customer level
customer_data = data.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'TotalSpend': 'sum',
    'InvoiceNo': 'nunique',  # Frequency
    'InvoiceDate': 'max'  # For Recency
}).reset_index()

# Calculate Frequency and Recency
customer_data.rename(columns={'InvoiceNo': 'Frequency'}, inplace=True)
customer_data['Recency'] = (data['InvoiceDate'].max() - customer_data['InvoiceDate']).dt.days

# Drop unnecessary columns
customer_data = customer_data[['CustomerID', 'Quantity', 'TotalSpend', 'Frequency', 'Recency']]

# Inspect the resulting dataframe
customer_data




C:\Users\saura\AppData\Local\Temp\ipykernel_20972\1254668315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TotalSpend'] = data['Quantity'] * data['UnitPrice']


,CustomerID,Quantity,TotalSpend,Frequency,Recency
0,-1.0,251587,1342813.57,3294,0
1,12346.0,0,0.00,2,325
2,12347.0,2458,4310.00,7,1
3,12348.0,2341,1797.24,4,74
4,12349.0,631,1757.55,1,18
...,...,...,...,...,...
4368,18280.0,45,180.60,1,277
4369,18281.0,54,80.82,1,180
4370,18282.0,98,176.60,3,7
4371,18283.0,1357,2045.53,16,3


In [17]:
from sklearn.preprocessing import StandardScaler

# Normalize the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_data[['Quantity', 'TotalSpend', 'Frequency', 'Recency']])


# Convert the scaled features back to a DataFrame
scaled_customer_data = pd.DataFrame(scaled_features, columns=['Quantity', 'TotalSpend', 'Frequency', 'Recency'])


In [18]:
scaled_customer_data

,Quantity,TotalSpend,Frequency,Recency
0,41.654716,61.217325,64.948717,-0.903471
1,-0.196381,-0.101563,-0.077493,2.322278
2,0.212503,0.095251,0.021271,-0.893545
3,0.193041,-0.019493,-0.037988,-0.168992
4,-0.091415,-0.021305,-0.097246,-0.724814
...,...,...,...,...
4368,-0.188895,-0.093316,-0.097246,1.845860
4369,-0.187398,-0.097872,-0.097246,0.883098
4370,-0.180079,-0.093498,-0.057741,-0.833993
4371,0.029354,-0.008154,0.199046,-0.873695
